<a href="https://colab.research.google.com/github/Leo-Lifeblood/Projects/blob/main/Lufthansa_Simulation_(Leonhard).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import statsmodels.stats.weightstats as smw
from tqdm.notebook import tqdm

class Passenger:
    def __init__(self, target, has_baggage):
        self.position = -1  # Start position outside the plane
        self.target = target  # Assigned seat (row number in this simplified model)
        self.baggage = has_baggage  # Boolean to indicate if the passenger has baggage
        self.baggage_delay = 3 if has_baggage else 0  # Delay steps if the passenger has baggage

    def __repr__(self):
        return f"Position: {self.position}, Target: {self.target}, Baggage: {self.baggage}"

    def __eq__(self, other):
        return self.target == other.target and self.position == other.position


class Plane:
    def __init__(self, num_rows, num_isles, boarding_strategy="random", imperfection=0.1, baggage_chance=0.5):
        self.num_rows = num_rows
        self.num_isles = num_isles
        self.imperfection = imperfection
        self.baggage_chance = baggage_chance  # Probability that a passenger has baggage
        self.strategy_map = {
            "back-to-front": 0,
            "front-to-back": 1,
            "random": 2,
            "lufthansa": 3,  # Placeholder for a potential real-world boarding method
            "WMA": 4,
            "steffen" : 5,
        }
        self.strategy = self.strategy_map.get(boarding_strategy.lower(), 2)  # Default to random
        self.reset_passengers()

    def shuffle(self):
        random.shuffle(self.passengers)

    def reset_passengers(self):
        self.passengers = [Passenger(i, random.random() < self.baggage_chance) for i in range(self.num_rows) for _ in range(self.num_isles)]
        self.shuffle()

    def organize_boarding_groups(self):
        if self.strategy == 0:
            self.passengers.sort(key=lambda x: -x.target)
        elif self.strategy == 1:
            self.passengers.sort(key=lambda x: x.target)
        elif self.strategy == 2:
            random.shuffle(self.passengers)
        elif self.strategy == 3:
            pre_boarding = [p for p in self.passengers if p.target < 5]  # Example: assuming first few rows for pre-boarding
            group1 = [p for p in self.passengers if 5 <= p.target < 10]  # Example: assuming rows for HON Circle, First Class
            group2 = [p for p in self.passengers if 10 <= p.target < 15]  # Business, Senators, Star Gold, Economy Flex
            #group3 = [p for p in self.passengers if 15 <= p.target < 20]  # Economy window seats
            #group4 = [p for p in self.passengers if 20 <= p.target < 25]  # Economy middle seats
            #group5 = [p for p in self.passengers if 25 <= p.target < 30]  # Economy aisle seats
            remainder = [p for p in self.passengers if 15 <= p.target < 30]
            remainder.sort(key=lambda x: -x.target)
            res = []
            [res.append(x) for x in remainder if x not in res]
            outc = []
            for h in range(self.num_isles):
                for i in res:
                    outc.append(Passenger(i.target, i.baggage))

            self.passengers = pre_boarding + group1 + group2 + outc
        elif self.strategy == 4:
            self.passengers.sort(key=lambda x: -x.target)
            res = []
            [res.append(x) for x in self.passengers if x not in res]
            outc = []
            for h in range(self.num_isles):
                for i in res:
                    outc.append(Passenger(i.target, i.baggage))
            self.passengers = outc
        elif self.strategy == 5:
            self.passengers.sort(key=lambda x: -x.target)
            res = []
            [res.append(x) for x in self.passengers if x not in res]
            outc = []
            for h in range(self.num_isles):
                for i in res:
                    outc.append(Passenger(i.target, i.baggage))
            outc = [p for p in outc if p.target % 2 == 0] + [p for p in outc if p.target % 2 == 1]
            self.passengers = outc

        # Introduce imperfection: shuffle a subset to simulate delays or misordering
        # Introduce realistic imperfections by perturbing positions
        for i in range(len(self.passengers)):
            if random.random() < self.imperfection:  # Probability of a perturbation
                perturb = random.randint(-3, 3)  # Allow shifts of up to 3 positions forward or backward
                new_index = max(0, min(len(self.passengers) - 1, i + perturb))  # Ensure new index is within bounds
                # Swap the current passenger with the one at new_index
                self.passengers[i], self.passengers[new_index] = self.passengers[new_index], self.passengers[i]
        subset_size = int(self.imperfection * len(self.passengers))
        subset = random.sample(self.passengers, subset_size)
        for p in subset:
            self.passengers.remove(p)
        random.shuffle(subset)
        self.passengers.extend(subset)

    def board_step(self):
        occupied_positions = {p.position for p in self.passengers if p.position >= 0}
        for p in self.passengers:
            if p.position == p.target:
                if p.baggage and p.baggage_delay > 0:
                    p.baggage_delay -= 1  # Decrement the delay counter
                    continue  # Skip moving this passenger this turn
            if p.position < p.target and p.position + 1 not in occupied_positions:
                if p.position >= 0:
                    occupied_positions.remove(p.position)
                p.position += 1
                occupied_positions.add(p.position)

        self.passengers = [p for p in self.passengers if not (p.position == p.target and p.baggage_delay == 0)]
        if -1 in occupied_positions and 0 not in occupied_positions:
            for p in self.passengers:
                if p.position == -1:
                    p.position = 0
                    break
        return not self.passengers


    def board(self):
        self.reset_passengers()
        self.organize_boarding_groups()
        time_taken = 0
        while not self.board_step():
            time_taken += 1
        return time_taken

def collect_data(num_samples, num_rows, num_isles, strategy, confidence=0.95):
    results = [Plane(num_rows, num_isles, strategy).board() for _ in tqdm(range(num_samples))]
    data = np.array(results)
    descr_stats = smw.DescrStatsW(data)
    mean_conf_interval = descr_stats.zconfint_mean(alpha=1-confidence)
    return {
        "strategy": strategy,
        "mean_time": round(data.mean(),3),
        "std_deviation": round(data.std(),3),
        "95%_conf_interval": (round(mean_conf_interval[0],3),round(mean_conf_interval[1],3))
    }

# Example usage:

results = collect_data(1000, 30, 6, "back-to-front")
print(results)
results = collect_data(1000, 30, 6, "random")
print(results)
results = collect_data(1000, 30, 6, "front-to-back")
print(results)
results = collect_data(1000, 30, 6, "lufthansa")
print(results)
results = collect_data(1000, 30, 6, "WMA")
print(results)
results = collect_data(1000, 30, 6, "steffen")
print(results)



  0%|          | 0/1000 [00:00<?, ?it/s]

{'strategy': 'back-to-front', 'mean_time': 374.978, 'std_deviation': 18.836, '95%_conf_interval': (373.81, 376.146)}


  0%|          | 0/1000 [00:00<?, ?it/s]

{'strategy': 'random', 'mean_time': 255.813, 'std_deviation': 10.009, '95%_conf_interval': (255.192, 256.434)}


  0%|          | 0/1000 [00:00<?, ?it/s]

...

{'strategy': 'lufthansa', 'mean_time': 274.513, 'std_deviation': 14.574, '95%_conf_interval': (273.609, 275.417)}


  0%|          | 0/1000 [00:00<?, ?it/s]

{'strategy': 'WMA', 'mean_time': 255.24, 'std_deviation': 10.489, '95%_conf_interval': (254.59, 255.89)}


  0%|          | 0/1000 [00:00<?, ?it/s]

{'strategy': 'steffen', 'mean_time': 222.149, 'std_deviation': 11.818, '95%_conf_interval': (221.416, 222.882)}


In [ ]:
import random
import numpy as np
import statsmodels.stats.weightstats as smw
from tqdm.notebook import tqdm

class Passenger:
    def __init__(self, target, has_baggage):
        self.position = -1  # Start position outside the plane
        self.target = target  # Assigned seat (row number in this simplified model)
        self.baggage = has_baggage  # Boolean to indicate if the passenger has baggage
        self.baggage_delay = 3 if has_baggage else 0  # Delay steps if the passenger has baggage

    def __repr__(self):
        return f"Position: {self.position}, Target: {self.target}, Baggage: {self.baggage}"

    def __eq__(self, other):
        return self.target == other.target and self.position == other.position


class Plane:
    def __init__(self, num_rows, num_isles, boarding_strategy="random", imperfection=0.1, baggage_chance=0.5):
        self.num_rows = num_rows
        self.num_isles = num_isles
        self.imperfection = imperfection
        self.baggage_chance = baggage_chance  # Probability that a passenger has baggage
        self.strategy_map = {
            "back-to-front": 0,
            "front-to-back": 1,
            "random": 2,
            "lufthansa": 3,  # Placeholder for a potential real-world boarding method
            "back-to-front-batched": 4,
            "steffen" : 5,
        }
        self.strategy = self.strategy_map.get(boarding_strategy.lower(), 2)  # Default to random
        self.reset_passengers()

    def shuffle(self):
        random.shuffle(self.passengers)

    def reset_passengers(self):
        self.passengers = [Passenger(i, random.random() < self.baggage_chance) for i in range(self.num_rows) for _ in range(self.num_isles)]
        self.shuffle()

    def organize_boarding_groups(self):
        if self.strategy == 0:
            self.passengers.sort(key=lambda x: -x.target)
        elif self.strategy == 1:
            self.passengers.sort(key=lambda x: x.target)
        elif self.strategy == 2:
            random.shuffle(self.passengers)
        elif self.strategy == 3:
            pre_boarding = [p for p in self.passengers if p.target < 5]  # Example: assuming first few rows for pre-boarding
            group1 = [p for p in self.passengers if 5 <= p.target < 10]  # Example: assuming rows for HON Circle, First Class
            group2 = [p for p in self.passengers if 10 <= p.target < 15]  # Business, Senators, Star Gold, Economy Flex
            group3 = [p for p in self.passengers if 15 <= p.target < 20]  # Economy window seats
            group4 = [p for p in self.passengers if 20 <= p.target < 25]  # Economy middle seats
            group5 = [p for p in self.passengers if 25 <= p.target < 30]  # Economy aisle seats

            self.passengers = pre_boarding + group1 + group2 + group3 + group4 + group5
        elif self.strategy == 4:
            self.passengers.sort(key=lambda x: -x.target)
            res = []
            [res.append(x) for x in self.passengers if x not in res]
            outc = []
            for h in range(self.num_isles):
                for i in res:
                    outc.append(Passenger(i.target, i.baggage))
            self.passengers = outc
        elif self.strategy == 5:
            self.passengers.sort(key=lambda x: -x.target)
            res = []
            [res.append(x) for x in self.passengers if x not in res]
            outc = []
            for h in range(self.num_isles):
                for i in res:
                    outc.append(Passenger(i.target, i.baggage))
            outc = [p for p in outc if p.target % 2 == 0] + [p for p in outc if p.target % 2 == 1]
            self.passengers = outc

        # Introduce imperfection: shuffle a subset to simulate delays or misordering
        # Introduce realistic imperfections by perturbing positions
        for i in range(len(self.passengers)):
            if random.random() < self.imperfection:  # Probability of a perturbation
                perturb = random.randint(-3, 3)  # Allow shifts of up to 3 positions forward or backward
                new_index = max(0, min(len(self.passengers) - 1, i + perturb))  # Ensure new index is within bounds
                # Swap the current passenger with the one at new_index
                self.passengers[i], self.passengers[new_index] = self.passengers[new_index], self.passengers[i]
        subset_size = int(self.imperfection * len(self.passengers))
        subset = random.sample(self.passengers, subset_size)
        for p in subset:
            self.passengers.remove(p)
        random.shuffle(subset)
        self.passengers.extend(subset)

    def board_step(self):
        occupied_positions = {p.position for p in self.passengers if p.position >= 0}
        for p in self.passengers:
            if p.position == p.target:
                if p.baggage and p.baggage_delay > 0:
                    p.baggage_delay -= 1  # Decrement the delay counter
                    continue  # Skip moving this passenger this turn
            if p.position < p.target and p.position + 1 not in occupied_positions:
                if p.position >= 0:
                    occupied_positions.remove(p.position)
                p.position += 1
                occupied_positions.add(p.position)

        self.passengers = [p for p in self.passengers if not (p.position == p.target and p.baggage_delay == 0)]
        if -1 in occupied_positions and 0 not in occupied_positions:
            for p in self.passengers:
                if p.position == -1:
                    p.position = 0
                    break
        return not self.passengers


    def board(self):
        self.reset_passengers()
        self.organize_boarding_groups()
        time_taken = 0
        while not self.board_step():
            time_taken += 1
        return time_taken

def collect_data(num_samples, num_rows, num_isles, strategy, confidence=0.95):
    results = [Plane(num_rows, num_isles, strategy).board() for _ in tqdm(range(num_samples))]
    data = np.array(results)
    descr_stats = smw.DescrStatsW(data)
    mean_conf_interval = descr_stats.zconfint_mean(alpha=1-confidence)
    return {
        "strategy": strategy,
        "mean_time": round(data.mean(),3),
        "std_deviation": round(data.std(),3),
        "95%_conf_interval": (round(mean_conf_interval[0],3),round(mean_conf_interval[1],3))
    }

# Example usage:

results = collect_data(1000, 30, 6, "back-to-front")
print(results)
results = collect_data(1000, 30, 6, "random")
print(results)
results = collect_data(1000, 30, 6, "front-to-back")
print(results)
results = collect_data(1000, 30, 6, "lufthansa")
print(results)
results = collect_data(1000, 30, 6, "back-to-front-batched")
print(results)
results = collect_data(1000, 30, 6, "steffen")
print(results)



  0%|          | 0/1000 [00:00<?, ?it/s]

{'strategy': 'back-to-front', 'mean_time': 374.814, 'std_deviation': 18.306, '95%_conf_interval': (373.679, 375.949)}


  0%|          | 0/1000 [00:00<?, ?it/s]

{'strategy': 'random', 'mean_time': 255.33, 'std_deviation': 10.478, '95%_conf_interval': (254.68, 255.98)}


  0%|          | 0/1000 [00:00<?, ?it/s]

...

{'strategy': 'lufthansa', 'mean_time': 318.984, 'std_deviation': 16.44, '95%_conf_interval': (317.965, 320.003)}


  0%|          | 0/1000 [00:00<?, ?it/s]

{'strategy': 'back-to-front-batched', 'mean_time': 213.592, 'std_deviation': 11.494, '95%_conf_interval': (212.879, 214.305)}


  0%|          | 0/1000 [00:00<?, ?it/s]

{'strategy': 'steffen', 'mean_time': 222.017, 'std_deviation': 12.152, '95%_conf_interval': (221.263, 222.771)}


In [ ]:
import random
import numpy as np
import statsmodels.stats.weightstats as smw
from tqdm.notebook import tqdm

class Passenger:
    def __init__(self, target, has_baggage):
        self.position = -1  # Start position outside the plane
        self.target = target  # Assigned seat (row number in this simplified model)
        self.baggage = has_baggage  # Boolean to indicate if the passenger has baggage
        self.baggage_delay = 3 if has_baggage else 0  # Delay steps if the passenger has baggage

    def __repr__(self):
        return f"Position: {self.position}, Target: {self.target}, Baggage: {self.baggage}"

    def __eq__(self, other):
        return self.target == other.target and self.position == other.position



class Plane:
    def __init__(self, num_rows, num_isles, boarding_strategy="random", imperfection=0.1, baggage_chance=0.5):
        self.num_rows = num_rows
        self.num_isles = num_isles
        self.imperfection = imperfection
        self.baggage_chance = baggage_chance  # Probability that a passenger has baggage
        self.strategy_map = {
            "back-to-front": 0,
            "front-to-back": 1,
            "random": 2,
            "lufthansa": 3,  # Placeholder for a potential real-world boarding method
            "WMA": 4,
            "back-to-front-batched": 4,
            "steffen" : 5,
        }
        self.strategy = self.strategy_map.get(boarding_strategy.lower(), 2)  # Default to random
        self.reset_passengers()

    def shuffle(self):
        random.shuffle(self.passengers)

    def reset_passengers(self):
        self.passengers = [Passenger(i, random.random() < self.baggage_chance) for i in range(self.num_rows) for _ in range(self.num_isles)]
        self.shuffle()

    def organize_boarding_groups(self):
        if self.strategy == 0:
            self.passengers.sort(key=lambda x: -x.target)
        elif self.strategy == 1:
            self.passengers.sort(key=lambda x: x.target)
        elif self.strategy == 2:
            random.shuffle(self.passengers)
        elif self.strategy == 3:
            pre_boarding = [p for p in self.passengers if p.target < 5]  # Example: assuming first few rows for pre-boarding
            group1 = [p for p in self.passengers if 5 <= p.target < 10]  # Example: assuming rows for HON Circle, First Class
            group2 = [p for p in self.passengers if 10 <= p.target < 15]  # Business, Senators, Star Gold, Economy Flex
            #group3 = [p for p in self.passengers if 15 <= p.target < 20]  # Economy window seats
            #group4 = [p for p in self.passengers if 20 <= p.target < 25]  # Economy middle seats
            #group5 = [p for p in self.passengers if 25 <= p.target < 30]  # Economy aisle seats
            remainder = [p for p in self.passengers if 15 <= p.target < 30]
            remainder.sort(key=lambda x: -x.target)
            res = []
            [res.append(x) for x in remainder if x not in res]
            outc = []
            for h in range(self.num_isles):
                for i in res:
                    outc.append(Passenger(i.target, i.baggage))

            self.passengers = pre_boarding + group1 + group2 + outc
        elif self.strategy == 4:
            self.passengers.sort(key=lambda x: -x.target)
            res = []
            [res.append(x) for x in self.passengers if x not in res]
            outc = []
            for h in range(self.num_isles):
                for i in res:
                    outc.append(Passenger(i.target, i.baggage))
            self.passengers = outc
        elif self.strategy == 5:
            self.passengers.sort(key=lambda x: -x.target)
            res = []
            [res.append(x) for x in self.passengers if x not in res]
            outc = []
            for h in range(self.num_isles):
                for i in res:
                    outc.append(Passenger(i.target, i.baggage))
            outc = [p for p in outc if p.target % 2 == 0] + [p for p in outc if p.target % 2 == 1]
            self.passengers = outc

        # Introduce imperfection: shuffle a subset to simulate delays or misordering
        # Introduce realistic imperfections by perturbing positions
        for i in range(len(self.passengers)):
            if random.random() < self.imperfection:  # Probability of a perturbation
                perturb = random.randint(-3, 3)  # Allow shifts of up to 3 positions forward or backward
                new_index = max(0, min(len(self.passengers) - 1, i + perturb))  # Ensure new index is within bounds
                # Swap the current passenger with the one at new_index
                self.passengers[i], self.passengers[new_index] = self.passengers[new_index], self.passengers[i]
        subset_size = int(self.imperfection * len(self.passengers))
        subset = random.sample(self.passengers, subset_size)
        for p in subset:
            self.passengers.remove(p)
        random.shuffle(subset)
        self.passengers.extend(subset)

    def board_step(self):
        occupied_positions = {p.position for p in self.passengers if p.position >= 0}
        for p in self.passengers:
            if p.position == p.target:
                if p.baggage and p.baggage_delay > 0:
                    p.baggage_delay -= 1  # Decrement the delay counter
                    continue  # Skip moving this passenger this turn
            if p.position < p.target and p.position + 1 not in occupied_positions:
                if p.position >= 0:
                    occupied_positions.remove(p.position)
                p.position += 1
                occupied_positions.add(p.position)

        self.passengers = [p for p in self.passengers if not (p.position == p.target and p.baggage_delay == 0)]
        if -1 in occupied_positions and 0 not in occupied_positions:
            for p in self.passengers:
                if p.position == -1:
                    p.position = 0
                    break
        return not self.passengers


    def board(self):
        self.reset_passengers()
        self.organize_boarding_groups()
        time_taken = 0
        while not self.board_step():
            time_taken += 1
        return time_taken

def collect_data(num_samples, num_rows, num_isles, strategy, confidence=0.95):
    results = [Plane(num_rows, num_isles, strategy).board() for _ in tqdm(range(num_samples))]
    data = np.array(results)
    descr_stats = smw.DescrStatsW(data)
    mean_conf_interval = descr_stats.zconfint_mean(alpha=1-confidence)
    return {
        "strategy": strategy,
        "mean_time": round(data.mean(),3),
        "std_deviation": round(data.std(),3),
        "95%_conf_interval": (round(mean_conf_interval[0],3),round(mean_conf_interval[1],3))
    }

# Example usage:
"""
results = collect_data(1000, 30, 6, "back-to-front")
print(results)
results = collect_data(1000, 30, 6, "random")
print(results)
results = collect_data(1000, 30, 6, "front-to-back")
print(results)
results = collect_data(1000, 30, 6, "lufthansa")
print(results)
"""
results = collect_data(1000, 30, 6, "WMA")
print(results)
results = collect_data(1000, 30, 6, "back-to-front-batched")
print(results)
results = collect_data(1000, 30, 6, "steffen")
print(results)



  0%|          | 0/1000 [00:00<?, ?it/s]

{'strategy': 'WMA', 'mean_time': 255.405, 'std_deviation': 10.809, '95%_conf_interval': (254.735, 256.075)}


  0%|          | 0/1000 [00:00<?, ?it/s]

{'strategy': 'back-to-front-batched', 'mean_time': 213.41, 'std_deviation': 11.954, '95%_conf_interval': (212.669, 214.151)}


  0%|          | 0/1000 [00:00<?, ?it/s]

{'strategy': 'steffen', 'mean_time': 221.931, 'std_deviation': 11.909, '95%_conf_interval': (221.193, 222.669)}
